#### AGENT
An agent take a self-determined, input-dependent sequence of steps before returning a user-facing output

First: Build an agent that has two tools: one to look things up online (Tavily), and one to look up specific data that we’ve loaded into a index using a retriever.

In [24]:
# Libraries

%pip install langchain
%pip install --upgrade --quiet  langchain-openai
%pip install langchainhub
%pip install BeautifulSoup4
%pip install faiss-cpu


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for langchainhub from https://files.pythonhosted.org/packages/1e/30/19859c841afcd54ee318709be0d456a8bd60d837005b80e99b35cd4195dc/langchainhub-0.1.15-py3-none-any.whl.metadata
  Obtaining dependency information for types-requests<3.0.0.0,>=2.31.0.2 from https://files.pythonhosted.org/packages/05/22/21c7918c9bb842faa92fd26108e9f669c3dee9b6b239e8f45dd5f673e6cf/types_requests-2.31.0.20240311-py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new r

In [18]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "ls__632ae1a420dc4b46b2894dd749f33b2e"

os.environ["TAVILY_API_KEY"] = "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"

os.environ["AZURE_OPENAI_ENDPOINT"] = "https://js-playground-01.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "43d936898d7645c3b3c1ebf294f04cd3"

#### Tavily Search engine tool

In [19]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()
search.invoke("what is the weather in SF")

[{'url': 'https://weather.com/weather/today/l/San+Francisco+CA+USCA0987:1:US',
  'content': "Today's and tonight's San Francisco, CA weather forecast, weather conditions and Doppler radar from The Weather Channel and Weather.com"},
 {'url': 'https://www.wunderground.com/forecast/us/ca/san-francisco',
  'content': 'San Francisco Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the San Francisco area.'},
 {'url': 'https://weather.com/weather/tenday/l/San Francisco CA USCA0987:1:US',
  'content': "Comfy & Cozy\nThat's Not What Was Expected\nOutside\n'No-Name Storms' In Florida\nGifts From On High\nWhat To Do For Wheezing\nSurviving The Season\nStay Safe\nAir Quality Index\nAir quality is considered satisfactory, and air pollution poses little or no risk.\n Health & Activities\nSeasonal Allergies and Pollen Count Forecast\nNo pollen detected in your area\nCold & Flu Forecast\nFlu risk is low in your

#### Retriever over data

In [30]:
# FAISS (Facebook AI Similarity Search) is a library that allows developers to quickly search for embeddings of documents that are similar to each other.

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain.docstore.document import Document

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

embeddings = AzureOpenAIEmbeddings(azure_deployment="ada-embedding", openai_api_version="2023-05-15")
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

retriever.get_relevant_documents("how to upload a dataset")[0]

# Document(page_content="dataset uploading.Once we have a dataset, how can we use it to test changes to a prompt or chain? The most basic approach is to run the chain over the data points and visualize the outputs. Despite technological advancements, there still is no substitute for looking at outputs by eye. Currently, running the chain over the data points needs to be done client-side. The LangSmith client makes it easy to pull down a dataset and then run a chain over them, logging the results to a new project associated with the dataset. From there, you can review them. We've made it easy to assign feedback to runs and mark them as correct or incorrect directly in the web app, displaying aggregate statistics for each test project.We also make it easier to evaluate these runs. To that end, we've added a set of evaluators to the open-source LangChain library. These evaluators can be specified when initiating a test run and will evaluate the results once the test run completes. If we’re being honest, most of", metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'})

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [31]:
# Tool

tools = [search, retriever_tool]

In [32]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_deployment="gpt4-32k",
)

In [33]:
from langchain import hub

# Get the prompt to use - you can modify this!

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [34]:
# Initialize an agent

from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [35]:
# AgentExecutor - repeatedly call the agent and execute tools

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [36]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [37]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyCookbookLangSmithOn this pageLangSmithIntroduction​LangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start​Tracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps​Check out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith suppo

{'input': 'how can langsmith help with testing?',
 'output': "LangSmith is a platform designed for building production-grade Language Models (LLM) applications. It provides several features that can assist with testing:\n\n1. **Debugging**: LangSmith allows you to debug your LLM applications, which is crucial during the testing phase to ensure that the application is functioning correctly.\n\n2. **Evaluation**: With LangSmith, you can evaluate your LLM applications. Evaluation is a necessary part of testing as it measures the performance and efficiency of your application.\n\n3. **Monitoring**: LangSmith's monitoring feature helps in tracking the performance of your application over time. This can be used to detect any issues or points of failure in your application during the testing phase.\n\n4. **Integration with LangChain**: LangSmith seamlessly integrates with LangChain, an open-source framework for building with LLMs. This allows for efficient testing of chains and intelligent ag